In [1]:
import torch
import torch.nn as nn

import torch.optim as optim

import torch.nn.functional as F

from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Subset

from torchinfo import summary

import torchvision.models as models

import os

#import models
from mobiface_like_v2 import MobiFace
from backbone import get_model
import model



from facenet_pytorch import MTCNN, InceptionResnetV1



import time

import numpy as np


import matplotlib.pyplot as plt

import time
import datetime

import random

import math
import torch
import numpy as np
from math import pi
from torch import nn

MobiFace(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (depthwise_conv): DepthwiseSeparableConv2d(
    (depthwise_conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
    (pointwise_conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): PReLU(num_parameters=1)
  )
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bottleneck_block1): BottleneckBlock(
    (conv1): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): PReLU(num_parameters=1)
    (depthwise_conv): DepthwiseSeparableConv2d(
      (depthwise_conv): Conv2d(32, 32, kernel_size=(3, 3), stri

c:\Users\mathe\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set a random seed for reproducibility
seed = 42
torch.manual_seed(seed)

# If using GPU, also set the seed for GPU
torch.cuda.manual_seed_all(seed)


In [3]:
#no logits
student = torch.load("KD_full_mobiFace_like_nologits_v2_4.pth")

In [4]:
student

MobiFace(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (depthwise_conv): DepthwiseSeparableConv2d(
    (depthwise_conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
    (pointwise_conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): PReLU(num_parameters=1)
  )
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bottleneck_block1): BottleneckBlock(
    (conv1): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): PReLU(num_parameters=1)
    (depthwise_conv): DepthwiseSeparableConv2d(
      (depthwise_conv): Conv2d(32, 32, kernel_size=(3, 3), stri

In [5]:
summary(student, (1,3,112,112))

Layer (type:depth-idx)                        Output Shape              Param #
MobiFace                                      [1, 512]                  --
├─Conv2d: 1-1                                 [1, 64, 56, 56]           1,728
├─BatchNorm2d: 1-2                            [1, 64, 56, 56]           128
├─PReLU: 1-3                                  [1, 64, 56, 56]           1
├─DepthwiseSeparableConv2d: 1-4               [1, 64, 56, 56]           --
│    └─Conv2d: 2-1                            [1, 64, 56, 56]           576
│    └─Conv2d: 2-2                            [1, 64, 56, 56]           4,096
│    └─BatchNorm2d: 2-3                       [1, 64, 56, 56]           128
│    └─PReLU: 2-4                             [1, 64, 56, 56]           1
├─BatchNorm2d: 1-5                            [1, 64, 56, 56]           128
├─PReLU: 1-6                                  [1, 64, 56, 56]           (recursive)
├─BottleneckBlock: 1-7                        [1, 64, 28, 28]           --
│  

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [7]:
# Load a pre-trained ResNet model

teacher = get_model(name="r50",fp16 = True)
teacher_v3_state_dict = torch.load('backbone.pth')
teacher.load_state_dict(teacher_v3_state_dict)
teacher.eval()

<All keys matched successfully>

KD Process

In [10]:
# Define data transformations
transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [11]:
# directory for test
root_dir= "C:\\Users\\mathe\\OneDrive\\Área de Trabalho\\master\\TFM\\dataset\\faces_webface_112x112\\images"


batch_size = 64

# Create ImageFolder dataset
dataset = datasets.ImageFolder(root=root_dir, transform=transform)

# Number of images for testing
num_test_images = 2000

# Total number of images
total_images = len(dataset)

# Indices of images for testing
test_indices = random.sample(range(total_images), num_test_images)

# Remaining indices for validation and training
remaining_indices = set(range(total_images)) - set(test_indices)


# Split remaining indices into validation and training sets
remaining_indices = list(remaining_indices)
random.shuffle(remaining_indices)


# Define the sizes of validation and training sets
val_size = int(0.30 * len(remaining_indices))


# Indices for validation and training sets
val_indices = remaining_indices[:val_size]
train_indices = remaining_indices[val_size:]


# Create Subset datasets
test_dataset = Subset(dataset, test_indices)
val_dataset = Subset(dataset, val_indices)
train_dataset = Subset(dataset, train_indices)


train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [12]:
student_model = student
teacher_model = teacher

teacher_model.to(device)
student_model.to(device)

MobiFace(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (depthwise_conv): DepthwiseSeparableConv2d(
    (depthwise_conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
    (pointwise_conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): PReLU(num_parameters=1)
  )
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bottleneck_block1): BottleneckBlock(
    (conv1): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): PReLU(num_parameters=1)
    (depthwise_conv): DepthwiseSeparableConv2d(
      (depthwise_conv): Conv2d(32, 32, kernel_size=(3, 3), stri

In [ ]:
class ArcFace(nn.Module):
    r"""Implement of ArcFace (https://arxiv.org/pdf/1801.07698v1.pdf):
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin
            cos(theta+m)
        Code origin : https://github.com/DavorJordacevic/IResNet-ArcFace-Knowledge-Distillation/blob/master/KD_Iresnet/utils/losses.py
        """
    def __init__(self, in_features, out_features, s=30, m=0.5):
        super(ArcFace, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

    def forward(self, input, label):
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        phi = cosine - self.m
        one_hot = torch.zeros_like(cosine)
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output

In [15]:

from torch.optim import lr_scheduler

start_time = time.time()
formatted_time = datetime.datetime.fromtimestamp(start_time).strftime('%Y-%m-%d %H:%M:%S')





student_model.train()


print(formatted_time)

# Define the criterion (ArcFace loss)
header = ArcFace(in_features=512, out_features=10575, s=50, m=0.15).to(device) #ArcFace(in_features=512, out_features=10575, s=30, m=0.5).to(device)

# Define the MSE loss
criterion_mse =  nn.MSELoss()

# Define the optimizer
optimizer = optim.Adam(student_model.parameters(), lr=0.001) 
#optimizer = optim.SGD(student_model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-5) 

scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0

    
    
    # Iterate over the batches in the training data loader
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()  # Clear accumulated gradients 

        # Forward pass
        with torch.no_grad():
            teacher_features = teacher_model(inputs)

        student_features = student_model(inputs)

        # Compute the ArcFace loss
        #thetas = header(student_features, labels)
        arcface_loss = 0 #nn.CrossEntropyLoss()(thetas, labels)

        # Compute the MSE loss
        mse_loss = criterion_mse(student_features, teacher_features)

        # Combine ArcFace and MSE losses
        alpha =  1 # Weight for MSE loss (you can adjust this)
        total_loss =   mse_loss * alpha + arcface_loss 


        # Backpropagation
        total_loss.backward()
        optimizer.step()
        

        running_loss += total_loss.item()

        # Print the current batch progress
        print(f"Batch [{i + 1}/{len(train_loader)}] loss {total_loss:.6f},learning rate: {optimizer.param_groups[0]['lr']:.4f},distillation loss {mse_loss* alpha:.6f},  running loss {running_loss / (i + 1):.4f}, ", end='\r')

    # Calculate the average loss for the epoch
    epoch_loss = running_loss / len(train_loader)
    
    # Print the average loss for the epoch
    print(f"\nEpoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}")

    scheduler.step()

    # Save the checkpoint
    checkpoint = {
        'epoch': epoch,
        'student_model_state_dict': student_model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch_loss': epoch_loss,
    }
    torch.save(checkpoint, f'checkpoint_epoch_kd_v2_2_featureOnly_{epoch}.pt')

# Measure execution time
end_time = time.time()
formatted_time = datetime.datetime.fromtimestamp(end_time).strftime('%Y-%m-%d %H:%M:%S')
print(formatted_time)
execution_time = end_time - start_time

# Save execution time
with open("execution_time_kd_feature_v2_2.txt", "w") as file:
    file.write(f"Execution time: {execution_time} seconds")

2024-03-24 23:16:01
Batch [5345/5345] loss 0.001161,learning rate: 0.0010,distillation loss 0.001161, ArcFace loss 0.0000, running loss 0.0015, 
Epoch [1/10], Loss: 0.0015
Batch [5345/5345] loss 0.000805,learning rate: 0.0009,distillation loss 0.000805, ArcFace loss 0.0000, running loss 0.0008, 
Epoch [2/10], Loss: 0.0008
Batch [5345/5345] loss 0.000659,learning rate: 0.0009,distillation loss 0.000659, ArcFace loss 0.0000, running loss 0.0006, 
Epoch [3/10], Loss: 0.0006
Batch [5345/5345] loss 0.000585,learning rate: 0.0009,distillation loss 0.000585, ArcFace loss 0.0000, running loss 0.0005, 
Epoch [4/10], Loss: 0.0005
Batch [5345/5345] loss 0.000528,learning rate: 0.0008,distillation loss 0.000528, ArcFace loss 0.0000, running loss 0.0005, 
Epoch [5/10], Loss: 0.0005
Batch [5345/5345] loss 0.000493,learning rate: 0.0008,distillation loss 0.000493, ArcFace loss 0.0000, running loss 0.0005, 
Epoch [6/10], Loss: 0.0005
Batch [5345/5345] loss 0.000463,learning rate: 0.0007,distillation l

In [16]:
torch.save(student_model, 'KD_full_mobiFace_like_feature_v2_2.pth') 
torch.save(student_model.state_dict(), 'KD_dict_mobiFace_like_feature_v2_2.pth') 